# **AP155 Lab Assignment**
## Module 5: Partial Differential Equations

_Instructions_: Answer each problem as completely as you can. Discuss **all** your answers as clearly and concisely as possible.

_Scoring Criteria_: 50% - *correctness of code*; 50% - *discussion of the code and results*. Maximum score is **100 points**.



### Student Information

_Full Name (Last Name, First Name)_: \Torero, William Klien
_Student No._:\ 2022-10739
_Section_: THV-TX-2

### Submission Information

_Date and Time Submitted (November 16, 2024 10:00PM)_:

**HONOR PLEDGE** I affirm that I have upheld the highest principles of honesty and integrity in my academic work and that this lab assignment is my own work.

**Sign here with your full name:** William Klien B. Torero

### Grading Information (c/o Lab Instructor)

TOTAL SCORE: <font color='red'>**100/100**</font>

Score breakdown:
* Problem 1 - <font color='red'>**100/100**</font>

<font color='red'>**signed GC Belinario**</font>

### PROBLEM 1
**The Schrodinger equation and the Crank-Nicolson method**

_Refer to Exercise 9.8 in the Newman text._ In this problem, you will use the Crank-Nicolson method to solve the full time-dependent Schrodinger equation and hence develop a picture of how a wavefunction evolves over time.

Consider an electron (mass $M = 9.109 \times 10^{-31}$ kg) in a box of length $L = 10^{-8}$ m. Suppose that at time $t = 0$ the wavefunction of the electron has the form

$$ \psi(x,0) = \exp\left[-\frac{(x-x_0)^2}{2\sigma^2}\right]e^{i\kappa x},$$
where $x_0 = \frac{L}{2}$, $\sigma = 1 \times 10^{-10}$ m, $\kappa = 5 \times 10^{10} {\rm m}^{-1}$,  and $\psi = 0$ on the walls at $x = 0$ and $x = L$.

1. Perform a single step of the Crank-Nicolson method for this electron, calculating the vector $\psi(t)$ of values of the wavefunction, given the initial wavefunction above and using $N = 1000$ spatial slices with $a = L/N$. Your program will have to perform the following steps. First, given the vector $\psi(0)$ at $t = 0$, you will have to multiply by the matrix $\bf{B}$ to get a vector $\bf{v} = \bf{B}\psi$. Because of the tridiagonal form of $\bf{B}$, this is fairly simple. The $i$th component of $\bf{v}$ is given by
$$ v_i = b_1\psi_i + b_2(\psi_{i+1} + \psi_{i-1}).$$

   You will also have to choose a value for the time-step $h$. A reasonable choice is $h = 10^{-18}$ s. *(30 pts.)*

2. Second you will have to solve the linear system ${\bf Ax}= {\bf v}$ for $\bf{x}$, which gives you the new value of $\psi$. You could do this using a standard linear equation solver like the function $\tt solve$ in numpy's $\tt linalg$. *(20 pts.)*

3. Once you have the code in place to perform a single step of the calculation, extend your program to perform repeated steps and hence solve for $\psi$ at a sequence of times a separation $h$ apart. Note that the matrix $\bf A$ is independent of time, so it doesn't change from one step to another. You can set up the matrix just once and then keep on reusing it for every step. *(30 pts.)*

4. Make an animation of the solution by displaying the real part of the wavefunction at each time-step. You can use the function rate from the package visual to ensure a smooth frame-rate for your animation-- see Section 3.5 on page 117 of the Newman text.

   Run your animation for a while and describe what you see. Write a few sentences explaining in physics terms what is going on in the system. *(20 pts.)*

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display

L = 1e-8 # Length of potential box (in meters)
m = 9.109e-31 # Mass of electron (in kilograms)
hbar = 1.0545718e-34 # Reduced Plancks constant
N = 1000 # Number of grid points
dx = L / N # Spatial step size
h = 1e-18 # Time step size
x = np.linspace(0, L, N) # Spatial grid points

# Constants of the given wave function
x0 = L / 2
sigma = 1e-10
k = 5e10

# The given initial wave function
psi_initial = np.exp(-(x - x0)**2 / (2 * sigma**2)) * np.exp(1j * k * x)

# Computing the Crank-Nicolson coefficient
a1 = 1 + 1j * hbar * h / (2 * m * dx**2) # Diagonal elements for matrix A
a2 = -1j * hbar * h / (2 * m * dx**2) / 2 # Non-diagonal elements for matrix A
b1 = 1 - 1j * hbar * h / (2 * m * dx**2) # Diagonal elements for matrix B
b2 = 1j * hbar * h / (2 * m * dx**2) / 2 # Non-diagonal elements for matrix B

# Initializing an empty/zero matrix for matrix A and B
A = np.zeros((N, N), dtype=complex)
B = np.zeros((N, N), dtype=complex)
# Iteration for each point in the grid
for i in range(N):
    A[i, i] = a1 # Changing the diagonal element of A
    B[i, i] = b1 # Changing the diagonal element of B
    if i > 0:
        A[i, i-1] = a2 # Lower diagonal elements of A
        B[i, i-1] = b2 # Lower diagonal elements of B
    if i < N - 1:
        A[i, i+1] = a2 # Upper diagonal elements of A
        B[i, i+1] = b2 # Upper diagonal elements of B

# Initializing a variable for the initial wave function
psi = psi_initial

# Defining a function to evolve the given wave function
def evolve(psi):
    v = B @ psi # Matrix multiplication of the right hand side vector
    psi_next = np.linalg.solve(A, v) # Solving the linear system of A and v
    return psi_next

# Initializing plot for the animation
fig, ax = plt.subplots()
line, = ax.plot(x, psi.real, color="blue", lw=1) # Plotting the real part of the wavefunction
ax.set_xlim(0, L) # Limits of x axis
ax.set_ylim(-1, 1) # Limits of y axis
ax.set_xlabel("x")
ax.set_ylabel("Real part of ψ")
ax.set_title("Time Evolution of the Wavefunction's Real Component")

# Defining a function that updates the wave function for animation frames
def update(frame):
    global psi
    psi = evolve(psi) # Updates the wavefunction base on the previous wavefunction
    line.set_ydata(psi.real) # Updating the  y value with the real part of wavefunction
    return line,

# Creating the animation
ani = FuncAnimation(fig, update, frames = 2000, interval=10) # Showing the evolutionn of wavefunction over each time step
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

Initially the wave packet spread over time, which reflects the uncertainty principle (momentum uncertainty increase, position uncertainty decrease). When it reaches the box's infinite walls, it reflects an inteference pattern which shows its wave-like nature. This is because if the initialized 0 wavefunctions at the boundaries.

<font color=red>Correctness of Code: 50/50 </font> \
<font color=red>Discussion of Code and Results: 50/50 </font>